In [2]:
import os
import sys
import csv
import random
import itertools
from operator import itemgetter
from collections import defaultdict
import numpy as np
import scipy
import scipy.spatial.distance
from numpy.linalg import svd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import utils

In [3]:
numDimen = 300
gloveFile = 'glove.6B.' + str(numDimen) + 'd.txt'
GLOVE = utils.glove2dict(os.path.join('glove.6B', gloveFile))

In [4]:
def randvec(n=50, lower=-0.5, upper=0.5):
    """Returns a random vector of length `n`"""
    return np.array([random.uniform(lower, upper) for i in range(n)])

def getGloveVec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE.get(w, randvec(n=numDimen))

def sentenceToGlove(words):
    result = [0] * numDimen
    for word in words:
        result = np.add(result, getGloveVec(word))
    return result

def getCosineSim(u, v):        
    """Cosine distance between 1d np.arrays `u` and `v`, which must have 
    the same dimensionality. Returns a float."""
    return scipy.spatial.distance.cosine(u, v)

def getL1Dist(u, v):
    return scipy.spatial.distance.cityblock(u, v)

def getL2Dist(u, v):
    return np.linalg.norm(u-v)

In [5]:
vocab = [0]

f = open('data/movie_50000.txt')
for line in f:
    vocab.append(line.rstrip('\n'))
f.close()

In [6]:
""" DEV SIDE """
sourceStrs = {}
goldTgtStrs = {}
targetStrs = []
indexes = []

f = open('data/dev.src_string.txt')
for i, line in enumerate(f):
    sourceStrs[i+1] = line.strip().split()
f.close()

f = open('data/dev.tgt_string.txt')
for i, line in enumerate(f):
    goldTgtStrs[i+1] = line.strip().split()
f.close()

f = open('data/index_dev.txt')
for line in f:
    indexes.append(int(line.rstrip('\n')))
f.close()

f = open('data/rearank_target_dev.txt')
for line in f:
    words = []
    vocabIndexes = line.strip().split()
    for index in vocabIndexes:
        words.append(vocab[int(index)])
    targetStrs.append(words)
f.close()

In [6]:
devSrcGloveDistances = []
devTgtGloveDistances = []
source = sourceStrs[indexes[0]]
sourceGlove = sentenceToGlove(source)
goldTgt = goldTgtStrs[indexes[0]]
goldTgtGlove = sentenceToGlove(goldTgt)

for i, tgtStr in enumerate(targetStrs):
    tgtStrGlove = sentenceToGlove(tgtStr)
    if source != sourceStrs[indexes[i]]:
        source = sourceStrs[indexes[i]]
        sourceGlove = sentenceToGlove(source)
        goldTgt = goldTgtStrs[indexes[i]]
        goldTgtGlove = sentenceToGlove(goldTgt)
    srcCosineSim = getCosineSim(sourceGlove, tgtStrGlove)
    devSrcGloveDistances.append(srcCosineSim)
    goldTgtCosineSim = getCosineSim(goldTgtGlove, tgtStrGlove)
    devTgtGloveDistances.append(goldTgtCosineSim)

In [7]:
np.savetxt("dev_src_glove_dist.txt", devSrcGloveDistances, fmt='%f')
np.savetxt("dev_tgt_glove_dist.txt", devTgtGloveDistances, fmt='%f')

In [7]:
devSrcL1Distances = []
devSrcL2Distances = []
devTgtL1Distances = []
devTgtL2Distances = []

source = sourceStrs[indexes[0]]
sourceGlove = sentenceToGlove(source)
goldTgt = goldTgtStrs[indexes[0]]
goldTgtGlove = sentenceToGlove(goldTgt)

for i, tgtStr in enumerate(targetStrs):
    tgtStrGlove = sentenceToGlove(tgtStr)
    if source != sourceStrs[indexes[i]]:
        source = sourceStrs[indexes[i]]
        sourceGlove = sentenceToGlove(source)
        goldTgt = goldTgtStrs[indexes[i]]
        goldTgtGlove = sentenceToGlove(goldTgt)
    devSrcL1Distances.append(getL1Dist(sourceGlove, tgtStrGlove))
    devSrcL2Distances.append(getL2Dist(sourceGlove, tgtStrGlove))
    devTgtL1Distances.append(getL1Dist(goldTgtGlove, tgtStrGlove))
    devTgtL2Distances.append(getL2Dist(goldTgtGlove, tgtStrGlove))

In [8]:
np.savetxt("dev_src_L1_dist.txt", devSrcL1Distances, fmt='%f')
np.savetxt("dev_src_L2_dist.txt", devSrcL2Distances, fmt='%f')
np.savetxt("dev_tgt_L1_dist.txt", devTgtL1Distances, fmt='%f')
np.savetxt("dev_tgt_L2_dist.txt", devTgtL2Distances, fmt='%f')

In [9]:
""" TEST SIDE """
sourceStrs = {}
goldTgtStrs = {}
targetStrs = []
indexes = []

f = open('data/test.src_string.txt')
for i, line in enumerate(f):
    sourceStrs[i+1] = line.strip().split()
f.close()

f = open('data/test.tgt_string.txt')
for i, line in enumerate(f):
    goldTgtStrs[i+1] = line.strip().split()
f.close()

f = open('data/index_valid.txt')
for line in f:
    indexes.append(int(line.rstrip('\n')))
f.close()

f = open('data/rearank_target_valid.txt')
for line in f:
    words = []
    vocabIndexes = line.strip().split()
    for index in vocabIndexes:
        words.append(vocab[int(index)])
    targetStrs.append(words)
f.close()

In [1]:
testSrcGloveDistances = []
testTgtGloveDistances = []
source = sourceStrs[indexes[0]]
sourceGlove = sentenceToGlove(source)
goldTgt = goldTgtStrs[indexes[0]]
goldTgtGlove = sentenceToGlove(goldTgt)

for i, tgtStr in enumerate(targetStrs):
    tgtStrGlove = sentenceToGlove(tgtStr)
    if source != sourceStrs[indexes[i]]:
        source = sourceStrs[indexes[i]]
        sourceGlove = sentenceToGlove(source)
        goldTgt = goldTgtStrs[indexes[i]]
        goldTgtGlove = sentenceToGlove(goldTgt)
    srcCosineSim = getCosineSim(sourceGlove, tgtStrGlove)
    testSrcGloveDistances.append(srcCosineSim)
    goldTgtCosineSim = getCosineSim(goldTgtGlove, tgtStrGlove)
    testTgtGloveDistances.append(goldTgtCosineSim)

NameError: name 'sourceStrs' is not defined

In [ ]:
np.savetxt("test_src_glove_dist.txt", testSrcGloveDistances, fmt='%f')
np.savetxt("test_tgt_glove_dist.txt", testTgtGloveDistances, fmt='%f')

In [10]:
testSrcL1Distances = []
testSrcL2Distances = []
testTgtL1Distances = []
testTgtL2Distances = []

source = sourceStrs[indexes[0]]
sourceGlove = sentenceToGlove(source)
goldTgt = goldTgtStrs[indexes[0]]
goldTgtGlove = sentenceToGlove(goldTgt)

for i, tgtStr in enumerate(targetStrs):
    tgtStrGlove = sentenceToGlove(tgtStr)
    if source != sourceStrs[indexes[i]]:
        source = sourceStrs[indexes[i]]
        sourceGlove = sentenceToGlove(source)
        goldTgt = goldTgtStrs[indexes[i]]
        goldTgtGlove = sentenceToGlove(goldTgt)
    testSrcL1Distances.append(getL1Dist(sourceGlove, tgtStrGlove))
    testSrcL2Distances.append(getL2Dist(sourceGlove, tgtStrGlove))
    testTgtL1Distances.append(getL1Dist(goldTgtGlove, tgtStrGlove))
    testTgtL2Distances.append(getL2Dist(goldTgtGlove, tgtStrGlove))

In [11]:
np.savetxt("test_src_L1_dist.txt", testSrcL1Distances, fmt='%f')
np.savetxt("test_src_L2_dist.txt", testSrcL2Distances, fmt='%f')
np.savetxt("test_tgt_L1_dist.txt", testTgtL1Distances, fmt='%f')
np.savetxt("test_tgt_L2_dist.txt", testTgtL2Distances, fmt='%f')